In [6]:
# Import libraries and API keys
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from pathlib import Path
from scipy import stats
from statistics import mean
from secrets import IEX_CLOUD_API_TOKEN

In [7]:
# Import NASDAQ list of stocks
stocks = pd.read_csv(
    Path('../project_1/Resources/nas.csv')
)

stocks

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,FB
4,TSLA
...,...
96,VRSN
97,DOCU
98,SWKS
99,SGEN


In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 97321505, 'calculationPrice': 'close', 'change': -1.63, 'changePercent': -0.00952, 'close': 167.8, 'closeSource': 'fioailfc', 'closeTime': 1664014955443, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 170.04, 'delayedPriceTime': 1699697403311, 'extendedChange': -0.8, 'extendedChangePercent': -0.00479, 'extendedPrice': 170.1, 'extendedPriceTime': 1723551951489, 'high': 176, 'highSource': 'len demi51pdyre ica tue', 'highTime': 1648337208896, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 173.62, 'iexCloseTime': 1711849113410, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 174.3, 'iexOpenTime': 1672582880777, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1645358551521, 'latestPrice': 171.9, 'latestSource': 'Close', 'latestTime': 'February 18, 2022', 'latestUpdate': 1694139537393, 'latestVolume': 84000858, 'low': 166.79, 'lowSource': ' mty1r ldeinee5iudce

In [10]:
# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

KeyError: 'AAPL'

In [14]:
# Create the DataFrame of financial metrics 

rv_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
        
rv_dataframe.drop(columns = 'Number of shares to buy', inplace=True)

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.613861
1      0.693069
2       0.50495
3      0.168317
4      0.950495
         ...   
96     0.792079
97     0.019802
98     0.366337
99      0.09901
100    0.108911
Name: PE Percentile, Length: 101, dtype: object
0      0.910891
1      0.772277
2       0.50495
3      0.227723
4      0.821782
         ...   
96     0.029703
97     0.960396
98      0.39604
99     0.316832
100    0.980198
Name: PB Percentile, Length: 101, dtype: object
0      0.663366
1      0.841584
2      0.158416
3      0.247525
4      0.712871
         ...   
96     0.930693
97     0.821782
98     0.415842
99     0.623762
100    0.643564
Name: PS Percentile, Length: 101, dtype: object
0      0.653465
1      0.712871
2       0.39604
3      0.118812
4      0.831683
         ...   
96     0.762376
97     0.960396
98     0.465347
99     0.089109
100    0.079208
Name: EV/EBITDA Percentile, Length: 101, dtype: object
0      0.851485
1      0.831683
2      0.138614
3      0.069307
4      0.930693
         ...   
9

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,168.20,28.57,0.613861,38.23,0.910891,7.48,0.663366,21.360399,0.653465,16.954242,0.851485,N/A
1,MSFT,297.99,32.13,0.693069,13.81,0.772277,11.89,0.841584,23.865071,0.712871,16.059687,0.831683,N/A
2,AMZN,3175.94,23.91,0.50495,5.84,0.50495,1.69,0.158416,12.287160,0.39604,3.667408,0.138614,N/A
3,FB,211.02,7.59,0.168317,2.45,0.227723,2.57,0.247525,4.371497,0.118812,2.591727,0.069307,N/A
4,TSLA,896.18,88.11,0.950495,14.78,0.821782,8.00,0.712871,44.445433,0.831683,30.374798,0.930693,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,218.29,39.93,0.792079,-16.53,0.029703,18.36,0.930693,26.510105,0.762376,21.019043,0.90099,N/A
97,DOCU,111.26,-198.51,0.019802,92.63,0.960396,11.24,0.821782,182.139211,0.960396,14.053676,0.792079,N/A
98,SWKS,143.06,16.54,0.366337,4.40,0.39604,4.40,0.415842,14.661106,0.465347,9.294231,0.524752,N/A
99,SGEN,124.51,-16.84,0.09901,3.60,0.316832,7.29,0.623762,-13.979740,0.089109,7.001295,0.386139,N/A


In [16]:

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,168.20,28.57,0.613861,38.23,0.910891,7.48,0.663366,21.360399,0.653465,16.954242,0.851485,0.738614
1,MSFT,297.99,32.13,0.693069,13.81,0.772277,11.89,0.841584,23.865071,0.712871,16.059687,0.831683,0.770297
2,AMZN,3175.94,23.91,0.50495,5.84,0.50495,1.69,0.158416,12.287160,0.39604,3.667408,0.138614,0.340594
3,FB,211.02,7.59,0.168317,2.45,0.227723,2.57,0.247525,4.371497,0.118812,2.591727,0.069307,0.166337
4,TSLA,896.18,88.11,0.950495,14.78,0.821782,8.00,0.712871,44.445433,0.831683,30.374798,0.930693,0.849505
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,218.29,39.93,0.792079,-16.53,0.029703,18.36,0.930693,26.510105,0.762376,21.019043,0.90099,0.683168
97,DOCU,111.26,-198.51,0.019802,92.63,0.960396,11.24,0.821782,182.139211,0.960396,14.053676,0.792079,0.710891
98,SWKS,143.06,16.54,0.366337,4.40,0.39604,4.40,0.415842,14.661106,0.465347,9.294231,0.524752,0.433663
99,SGEN,124.51,-16.84,0.09901,3.60,0.316832,7.29,0.623762,-13.979740,0.089109,7.001295,0.386139,0.30297


In [28]:
# Sort the DataFrame by Robust Value score
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:20]
rv_dataframe.reset_index(drop = True, inplace = True)

rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,CMCSA,48.72,8.00,0.178218,1.1000,0.079208,0.9312,0.039604,5.284812,0.148515,1.622127,0.009901,0.091089
1,INTC,45.77,4.76,0.148515,1.0012,0.069307,1.1800,0.084158,2.974535,0.09901,2.306467,0.059406,0.092079
2,WBA,48.60,6.29,0.158416,1.5500,0.118812,0.3234,0.009901,7.572049,0.247525,1.762751,0.019802,0.110891
3,BIIB,219.23,10.09,0.207921,1.4400,0.108911,1.4000,0.118812,5.270665,0.128713,2.222930,0.039604,0.120792
4,REGN,620.70,4.34,0.138614,1.7400,0.148515,2.0700,0.19802,3.325108,0.108911,2.077024,0.029703,0.124752
5,CTSH,89.60,11.00,0.257426,1.8900,0.158416,1.2400,0.09901,6.213853,0.158416,3.080379,0.09901,0.154455
6,FB,211.02,7.59,0.168317,2.4500,0.227723,2.5700,0.247525,4.371497,0.118812,2.591727,0.069307,0.166337
7,SIRI,6.03,9.77,0.19802,-4.6600,0.049505,1.4300,0.128713,8.002234,0.287129,4.374956,0.207921,0.174257
8,CHTR,622.00,12.84,0.306931,3.8400,0.346535,1.0500,0.059406,6.959652,0.19802,2.846453,0.089109,0.2
9,TMUS,126.30,26.39,0.584158,1.1300,0.089109,0.9852,0.049505,6.648465,0.178218,3.443969,0.118812,0.20396


In [21]:
writer = pd.ExcelWriter('value_strat.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()